In [2]:
from ib_insync import IB, Stock, util, Order
import pandas as pd
import talib
import numpy as np
import time
import nest_asyncio

# Start connection

In [3]:
# Apply nest_asyncio to allow asyncio in Jupyter Notebook
nest_asyncio.apply()

# Connect to Interactive Brokers
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

API connection failed: ConnectionRefusedError(10061, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 10061] Connect call failed ('127.0.0.1', 7497)

# Set up parameters

In [ ]:
# Parameters
SYMBOL = 'AAPL'  # Replace with desired ticker
TIMEFRAME = '1 min'  # 1-minute timeframe
LOOKBACK = 100  # Number of candles to fetch

# Strategy parameters
dollar_stop = 1000
atr_multiplier_loss = 1.0
atr_multiplier_profit = 3.4
rsi_period = 10
atr_period = 14
high_period = 90
cooldown_long_win = 15
cooldown_long_loss = 5

# Retrieve data

In [ ]:
# Fetch historical data
contract = Stock(SYMBOL, 'SMART', 'USD')
bars = ib.reqHistoricalData(
    contract,
    endDateTime='',
    durationStr='2 D',
    barSizeSetting=TIMEFRAME,
    whatToShow='TRADES',
    useRTH=True
)

# Convert to DataFrame
df = util.df(bars)
df['Date'] = pd.to_datetime(df['date'])
df.set_index('Date', inplace=True)

# Ensure we have at least `LOOKBACK` rows
if len(df) < LOOKBACK:
    raise ValueError(f"Not enough data available. Expected {LOOKBACK}, got {len(df)}")

# Add indicators
df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=atr_period)
df['RSI'] = talib.RSI(df['close'], timeperiod=rsi_period)
df['HighestClose'] = df['close'].rolling(window=high_period).max()

# Track state
sl, tp = None, None
cooldown = 0

Error 162, reqId 373: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


Error: 'NoneType' object has no attribute 'iloc'


Error 162, reqId 374: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


Error: 'NoneType' object has no attribute 'iloc'


Error 162, reqId 375: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


...

Error: 'NoneType' object has no attribute 'iloc'


Error 162, reqId 382: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


Error: 'NoneType' object has no attribute 'iloc'


Error 162, reqId 383: Historical Market Data Service error message:HMDS query returned no data: AAPL@SMART Trades, contract: Stock(symbol='AAPL', exchange='SMART', currency='USD')


Error: 'NoneType' object has no attribute 'iloc'


# Execute strategy

In [ ]:
def execute_strategy():
    """
    Function to execute the ATR-Enhanced RSI Breakout strategy in Jupyter Notebook.
    This function should be run repeatedly using a loop or cell execution.
    """
    global sl, tp, cooldown, df

    # Fetch the latest bar
    bars = ib.reqHistoricalData(
        contract,
        endDateTime='',
        durationStr='1 D',
        barSizeSetting=TIMEFRAME,
        whatToShow='TRADES',
        useRTH=True
    )
    latest = util.df(bars).iloc[-1]

    # Add latest bar to DataFrame
    new_row = pd.DataFrame([latest], index=[latest['date']])
    df = pd.concat([df, new_row]).tail(LOOKBACK)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=atr_period)
    df['RSI'] = talib.RSI(df['close'], timeperiod=rsi_period)
    df['HighestClose'] = df['close'].rolling(window=high_period).max()

    # Ensure cooldown
    if cooldown > 0:
        print(f"Cooldown active: {cooldown} bars remaining.")
        cooldown -= 1
        return

    # Trade logic
    close = df['close'].iloc[-1]
    highest_close = df['HighestClose'].iloc[-1]
    rsi = df['RSI'].iloc[-1]
    atr = df['ATR'].iloc[-1]

    stop_loss_level = dollar_stop / close
    atr_stop = atr_multiplier_loss * atr
    atr_profit = atr_multiplier_profit * atr

    # Long Entry
    if close >= highest_close and rsi > 50:
        sl = close - max(stop_loss_level, atr_stop)
        tp = close + atr_profit
        print(f"Entering LONG: Close={close}, SL={sl}, TP={tp}")

        # Create a market order
        order = Order(action='BUY', totalQuantity=100, orderType='MKT')
        ib.placeOrder(contract, order)

    # Trade Exit Logic
    if sl and tp:
        if close <= sl:
            print(f"Stop-loss hit: Close={close}, SL={sl}")
            
            # Create a market sell order to exit position
            exit_order = Order(action='SELL', totalQuantity=100, orderType='MKT')
            ib.placeOrder(contract, exit_order)
            cooldown = cooldown_long_loss
            sl, tp = None, None
        elif close >= tp:
            print(f"Take-profit hit: Close={close}, TP={tp}")
            
            # Create a market sell order to exit position
            exit_order = Order(action='SELL', totalQuantity=100, orderType='MKT')
            ib.placeOrder(contract, exit_order)
            cooldown = cooldown_long_win
            sl, tp = None, None


# Run the strategy repeatedly
while True:
    try:
        execute_strategy()
    except Exception as e:
        print(f"Error: {e}")
    time.sleep(60)  # Wait for 1 minute before the next iteration

In [ ]:
ib.disconnect()